In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ecephys_analyses.on_off import run_on_off_detection, get_subregions

from joblib import Parallel, delayed

In [ ]:
condition_groups = [
    ('Allan', 'SD'),
]

data_conditions = [
    ('Doppio', 'sleep-homeostasis-2h_imec1', 'ks2_5_catgt_Th=12-10_lam=50_8s-batches_postpro_1', 'cortex')
#     ('Doppio', 'sleep-homeostasis-2h_imec0'),
]
# subject = 'Doppio'
# condition = 'sleep-homeostasis-2h_imec1'

subregions_params = {
    'binsize': 20,  # um
    'smoothing_sd': 3, # units of binsize
    'prominence': 0.0, # Percent of maximal peak
}

good_only = False

pool = True

state = 'N2'

detection_condition = 'on_off_threshold_1'

n_jobs = 1

In [ ]:
all_args = []
for (
    subject, condition, sorting_condition, region,
) in data_conditions:
    subregions = get_subregions(
        subject,
        condition,
        sorting_condition,
        region,
        subregions_params,
    )

    for i, (subregion_name, subregion_depths) in enumerate(subregions.items()):
        all_args.append((subject, condition, sorting_condition, subregion_name, subregion_depths))
print(all_args)

In [ ]:
if n_jobs == 1:
    for (
        subject, condition, sorting_condition, subregion_name, subregion_depths
    ) in all_args:

        print(subject, condition, sorting_condition, subregion_name, subregion_depths)
        
        on_off_df = run_on_off_detection(
            subject,
            condition,
            detection_condition,
            sorting_condition=sorting_condition,
            region=None,
            region_name=subregion_name,
            depth_interval=subregion_depths,
            state=state,
            good_only=good_only,
            pool=pool,
        )
else:
    
    parallel = Parallel(
        n_jobs=n_jobs,
        backend='multiprocessing',
    )(
        delayed(run_on_off_detection)(
            subject,
            condition,
            detection_condition,
            sorting_condition=sorting_condition,
            region=None,
            region_name=subregion_name,
            depth_interval=subregion_depths,
            state=state,
            good_only=good_only,
            pool=pool,
        ) for (
            subject, condition, sorting_condition, subregion_name, subregion_depths
        ) in all_args
    )